<a href="https://colab.research.google.com/github/baiyunming/I2DL-WS2020/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
import torch
from google.colab import drive
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm.notebook import tqdm

In [ ]:
drive.mount('/content/drive')

In [3]:
%cd drive/MyDrive/ObjectDetection/

/content/drive/MyDrive/ObjectDetection


In [4]:
from model import YOLO_Resnet
from loss import YoloLoss
from animal_dataset import AnimalDataset
from utils import non_max_suppression, mean_average_precision, get_list_boxes

In [5]:
train_dataset = AnimalDataset("/content/drive/MyDrive/ObjectDetection/animal_dataset/train_40.csv")

In [6]:
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = YOLO_Resnet().to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [10]:
optimizer = optim.Adam(model.parameters(), lr=2e-5)

In [11]:
def train(model, optimizer, image, label):
    model.train()
    batch_size = image.shape[0]

    criterion = YoloLoss()

    optimizer.zero_grad()

    input = image.to(device)
    target = label.to(device) 
    
    pred = model(input)
    loss = criterion(pred, target)/batch_size
    
    loss.backward()
       
    optimizer.step()

    return loss.item()

In [36]:
max_epoch = 10
train_loss = 0

for epoch in tqdm(range(max_epoch)):   
  # Iterate over the train_dataloader
    with tqdm(total=len(train_dataloader)) as pbar:
        for idx, [image, label] in enumerate(train_dataloader):
            curr_loss = train(model, optimizer, image, label)
            print(curr_loss)
            train_loss += curr_loss / len(train_dataloader)
            pbar.update(1)

9.577585220336914
8.470080375671387
8.862536430358887
10.966465950012207
8.231437683105469



8.850452423095703
13.116381645202637
8.176490783691406
6.978919982910156
6.569538593292236



8.347347259521484
6.339571952819824
8.205866813659668
11.715221405029297
9.648219108581543



7.187539100646973
10.660146713256836
8.5904541015625
8.376787185668945
8.570833206176758



7.541049957275391
9.73336410522461
8.944196701049805
11.971410751342773
9.617987632751465



14.105900764465332
9.615365028381348
7.737657070159912
8.10473918914795
6.338389873504639



12.483569145202637
10.360116004943848
8.392197608947754
6.796895980834961
8.390137672424316



5.782731056213379
7.995673179626465
7.390386581420898
11.54092788696289
9.681551933288574



9.355281829833984
6.349513530731201
10.795663833618164
6.785236358642578
6.5818190574646



12.497722625732422
7.817997932434082
6.38516902923584
9.639626502990723
9.584739685058594




In [37]:
model.eval()
all_predict_boxes = []
all_true_boxes = []
frame_idx = 0
iou_threshold = 0.2
confidence_threshold = 0.3


for batch_idx, (image, labels) in enumerate(train_dataloader):
  with torch.no_grad():
    prediction = model(image.to(device))
  
  predict_boxes_batch = get_list_boxes(prediction, S=5)
  true_boxes_batch = get_list_boxes(labels.to(device), S=5)
  batch_size = prediction.shape[0]

  for i in range(batch_size):
    predict_boxes_frame = predict_boxes_batch[i]
    nms_predict_boxes_frame = non_max_suppression(predict_boxes_frame,iou_threshold,confidence_threshold)

    for j in range(len(nms_predict_boxes_frame)):
      all_predict_boxes.append([frame_idx] + nms_predict_boxes_frame[j])

    true_boxes_frame = true_boxes_batch[i]
    for k in range(len(true_boxes_frame)):
      if true_boxes_frame[k][1] > confidence_threshold:
        all_true_boxes.append([frame_idx] + true_boxes_frame[k])
    frame_idx = frame_idx + 1

torch.Size([8, 25, 6])
torch.Size([8, 25, 6])
torch.Size([8, 25, 6])
torch.Size([8, 25, 6])
torch.Size([8, 25, 6])
torch.Size([8, 25, 6])
torch.Size([8, 25, 6])
torch.Size([8, 25, 6])
torch.Size([7, 25, 6])
torch.Size([7, 25, 6])


In [38]:
mean_average_precision(all_predict_boxes, all_true_boxes)

tensor(0.4757)